In [1]:
import os
import tensorflow as tf
import numpy as np


In [2]:
# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()
print(cwd)

/Users/julianjimenez/Desktop/ANND/Project


## Dataset Creation

In [3]:
# ImageDataGenerator
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = False

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255)
    
#Create validation ImageDataGenerator Object
valid_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [4]:
dataset_dir = os.path.join(cwd, 'Classification_Dataset')

#Batch Size
bs = 64
# img shape
img_h = 256
img_w = 256

num_classes = 20

classes = ['owl',              # 0
           'galaxy',           # 1
           'lightning',        # 2
           'wine-bottle',      # 3
           't-shirt',          # 4
           'waterfall',        # 5
           'sword',            # 6
           'school-bus',       # 7
           'calculator',       # 8
           'sheet-music',      # 9
           'airplanes',        # 10
           'lightbulb',        # 11
           'skyscraper',       # 12
           'mountain-bike',     # 13
           'fireworks',        # 14
           'computer-monitor', # 15
           'bear',             # 16
           'grand-piano',      # 17
           'kangaroo',         # 18
           'laptop']           # 19

In [5]:
# Training
training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               batch_size=bs,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)

Found 1554 images belonging to 20 classes.


In [15]:
len(train_gen)

25

In [6]:
#Create Dataset objects

train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
train_dataset = train_dataset.repeat()

In [7]:
train_dataset

<DatasetV1Adapter shapes: ((None, 256, 256, 3), (None, 20)), types: (tf.float32, tf.float32)>

In [8]:
valid_dataset = train_dataset.take(311)
valid_dataset.repeat()

<DatasetV1Adapter shapes: ((None, 256, 256, 3), (None, 20)), types: (tf.float32, tf.float32)>

In [9]:
# Keras Model subclassing 
# -----------------------

# Please note that there are many ways to implement a CNN
# Here subclassing is used just for teaching purposes, but you
# can simply write every single layer as usual using Model or Sequential..

# Create convolutional block
class ConvBlock(tf.keras.Model):
    def __init__(self, num_filters):
        super(ConvBlock, self).__init__()
        self.conv2d = tf.keras.layers.Conv2D(filters=num_filters,
                                             kernel_size=(3, 3),
                                             strides=(1, 1), 
                                             padding='same')
        self.activation = tf.keras.layers.ReLU()  # we can specify the activation function directly in Conv2D
        self.pooling = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
        
    def call(self, inputs):
        x = self.conv2d(inputs)
        x = self.activation(x)
        x = self.pooling(x)
        return x

In [10]:
# Create Model
# ------------

depth = 5
start_f = 8
num_classes = 20

class CNNClassifier(tf.keras.Model):
    def __init__(self, depth, start_f, num_classes):
        super(CNNClassifier, self).__init__()
        
        self.feature_extractor = tf.keras.Sequential()
    
        for i in range(depth):
            self.feature_extractor.add(ConvBlock(num_filters=start_f))
            start_f *= 2
            
        self.flatten = tf.keras.layers.Flatten()
        self.classifier = tf.keras.Sequential()
        self.classifier.add(tf.keras.layers.Dense(units=512, activation='relu'))
        self.classifier.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
        
    def call(self, inputs):
        x = self.feature_extractor(inputs)
        x = self.flatten(x)
        x = self.classifier(x)
        return x
    
# Create Model instance
model = CNNClassifier(depth=depth,
                      start_f=start_f,
                      num_classes=num_classes)
# Build Model (Required)
model.build(input_shape=(None, img_h, img_w, 3))

In [11]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [12]:
# Visualize created model as a table
model.feature_extractor.summary()

# Visualize initialized weights
model.weights


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_block (ConvBlock)       multiple                  224       
_________________________________________________________________
conv_block_1 (ConvBlock)     multiple                  1168      
_________________________________________________________________
conv_block_2 (ConvBlock)     multiple                  4640      
_________________________________________________________________
conv_block_3 (ConvBlock)     multiple                  18496     
_________________________________________________________________
conv_block_4 (ConvBlock)     multiple                  73856     
Total params: 98,384
Trainable params: 98,384
Non-trainable params: 0
_________________________________________________________________


[<tf.Variable 'sequential/conv_block/conv2d/kernel:0' shape=(3, 3, 3, 8) dtype=float32, numpy=
 array([[[[ 1.87292695e-02, -6.66571707e-02,  4.01918888e-02,
           -1.26130164e-01, -3.43003571e-02,  1.99994743e-01,
            3.55525315e-03, -6.95133060e-02],
          [ 1.37295932e-01,  4.19998467e-02,  1.32354498e-01,
           -8.24365020e-03,  4.30276692e-02, -2.35006183e-01,
           -3.16811651e-02,  2.35468864e-01],
          [ 2.36862689e-01, -1.74060702e-01,  2.43510842e-01,
            1.38533324e-01, -1.37798309e-01, -5.41950762e-03,
            6.55457079e-02,  3.13819945e-02]],
 
         [[-4.74313945e-02, -4.98768538e-02,  8.99522901e-02,
            1.34971857e-01, -2.68238485e-02, -2.18192115e-01,
           -1.25084400e-01, -9.58562642e-02],
          [-8.99255723e-02, -1.51148498e-01,  1.60095960e-01,
            4.54548597e-02, -6.18434399e-02,  1.69046938e-01,
           -1.27805948e-01,  1.58601552e-01],
          [-1.66178837e-01, -1.14610642e-01, -4.2898

In [13]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [16]:
import os
from datetime import datetime

# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = False
if early_stop:
    es_callback = tf.keras.callback.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)


model.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=5, 
          callbacks=callbacks)

Train for 25 steps, validate for 5 steps
Epoch 1/100
25/25 [==============================] - 26s 1s/step - loss: 2.0808 - accuracy: 0.3900 - val_loss: 1.8305 - val_accuracy: 0.4281
Epoch 2/100
25/25 [==============================] - 27s 1s/step - loss: 1.5634 - accuracy: 0.5206 - val_loss: 1.5216 - val_accuracy: 0.5531
Epoch 3/100
25/25 [==============================] - 39s 2s/step - loss: 1.4690 - accuracy: 0.5624 - val_loss: 1.2788 - val_accuracy: 0.6031
Epoch 4/100
10/25 [===========>..................] - ETA: 22s - loss: 1.1517 - accuracy: 0.6472

KeyboardInterrupt: 